In [1]:
import sys
import time
from itertools import permutations

board = []
depth = 0

In [ ]:
def import_board():
    input_path = sys.argv[1]
    input_file = open("C:\Users\salehin\boards.txt", "r")
    split_list = input_file.read().split('\n')
    # reading data
    dimensions = []   # 10*10 dimention
    rows_array = []   # number of row 
    cols_array = []   #number of columns 
 

In [ ]:

# validates finished boards
def final_validation(rows, cols):
    original_cols = []
    single_col = []
    for x in range(len(board[0])):
        prev_white = True
        for i in range(len(board)):
            if board[i][x] == 1:
                prev_white = True
            elif board[i][x] == 2:
                if prev_white:
                    single_col.append(1)
                else:
                    single_col[len(single_col)-1] += 1
                prev_white = False
        original_cols.append(single_col.copy())
        single_col.clear()
    if cols != original_cols:
        return False

    original_rows = []
    single_row = []
    for x in range(len(board)):
        prev_white = True
        for i in range(len(board[0])):
            if board[x][i] == 1:
                prev_white = True
            elif board[x][i] == 2:
                if prev_white:
                    single_row.append(1)
                else:
                    single_row[len(single_row)-1] += 1
                prev_white = False
        original_rows.append(single_row.copy())
        single_row.clear()
    return False if rows != original_rows else True

In [ ]:

# find first row with an empty cell otherwise returns board height
def find_empty():
    for i in range(len(board)):
        if board[i][0] == 0:
            return i
    return len(board)


# n is the integer to partition, k is the length of partitions, l is the min partition element size
def partition(n, k, l=1):
    if k < 1:
        raise StopIteration
    if k == 1:
        if n >= l:
            yield (n,)
        return
    for i in range(l, n + 1):
        for result in partition(n - i, k - 1, i):
            yield (i,) + result

In [ ]:
# all possible combinations of row with index idx
def row_permutations(board_length, rows, idx):
    row_elems = len(rows[idx])
    partitions_elems_size = row_elems + 1
    first_elem = True
    row_elems_size = 0
    for x in range(len(rows[idx])):
        if not first_elem:
            row_elems_size += 1
        row_elems_size += rows[idx][x]
    partitions_elems = board_length - row_elems_size - row_elems + 1

    partitions = []
    for i in partition(partitions_elems, partitions_elems_size, 0):
        partitions.append(i)

    partition_permutations_set = set()
    for i in partitions:
        partition_permutations_set.update(permutations(i))

    partition_permutations_list = [list(ele) for ele in partition_permutations_set]
    partition_permutations_list.sort()

    possible_rows = []
    for perm in partition_permutations_list:
        poss_row = []
        first_block = True
        for n in range(row_elems):
            for cell in range(perm[n]):
                poss_row.append(1)
            if not first_block:
                poss_row.append(1)
            first_block = False
            for cell in range(rows[idx][n]):
                poss_row.append(2)
        for cell in range(perm[len(perm) - 1]):
            poss_row.append(1)
        possible_rows.append(poss_row)

    return possible_rows

In [ ]:
# recursive function that finds nonogram's solution
def solve(rows, cols):
    global depth
    depth += 1
    if find_empty() == len(board):
        return True
    for idx in range(find_empty(), len(board)):
        for row in row_permutations(len(board[0]), rows, idx):
            board[idx] = row
            if validate_vertically(cols, idx):
                if solve(rows, cols):
                    return True
                for y in range(len(board[idx])):
                    board[idx][y] = 0
            else:
                for y in range(len(board[idx])):
                    board[idx][y] = 0
        return False

In [ ]:
# main function
if __name__ == '__main__':
    board, rows, cols = import_board()
    start_time = time.time()
    solve(rows, cols)
    end_time = time.time() - start_time
    print("*")
    output_path = sys.argv[1]
    output_path = output_path.strip('./boards/')
    output_path = "./results/out_" + output_path
    if final_validation(rows, cols):
        with open(output_path, 'a+') as output_file:
            print("Recursion calls: %d" % depth, file=output_file)
            print(file=output_file)
        output_file.close()
        print("output", output_path)
    else:
        print("Nonogram", sys.argv[1].strip('./boards/'), "*")
    print()